# Traveling Salesman Problem

We are given a [complete graph](https://en.wikipedia.org/wiki/Complete_graph) which means that every pair of distinct vertices is connected by a unique edge. Each edge has traveling cost associated with it. We need to find a shortest route that goes throught all the nodes and comes back to the original node.

## Representation

We can represent traveling costs as a simple matrix.

In [1]:
i = 'i'
m =  [
    [i, 3, 1, 8],
    [3, i, 4, 4],
    [1, 4, i, 7],
    [8, 4, 7, i],
]

This represents 4 nodes (0 through 3) where cost of traveling from 0 to 3 is

In [2]:
m[0][3]

8

## Brute force solution

In [3]:
from itertools import permutations

def total_cost(matrix, route):
    # we are going from node a to node b in each step
    a = list(route)
    b = a[1:] + [a[0]]

    # calculate the route cost
    cost = 0
    for ii, jj in zip(a, b):
        cost += matrix[ii][jj]
    return cost


def brute_force(matrix):
    current_min = pow(10, 10) # should be at least bigger than min route
    current_route = []
    for route in permutations(range(len(matrix))):
        route_cost = total_cost(matrix, route)
        # check if we got new min route
        if route_cost < current_min:
            current_min = route_cost
            current_route = route
    
    return current_route, current_min

In [4]:
route, distance = brute_force(m)

In [5]:
print('Found route:', route, 'with distance:', distance)

Found route: (0, 1, 3, 2) with distance: 15


## Greedy solution
What if we just always pick the shortest distance from all available routes in a current node. We start with first node and pick the cheapest node that we haven't picked yet.

In [6]:
def get_min_node(nodes, visited_set):
    min_cost = pow(10, 10)
    node_position = None
    for position, cost in enumerate(nodes):
        # make sure we haven't visited the node yet, we are not trying to go to
        # the same node, and current cost is the minimum that we've seen so far
        if position in visited_set or isinstance(cost, str) or cost >= min_cost:
            continue

        min_cost = cost
        node_position = position

    return node_position

def greedy(matrix, visited=set(), route=[0]):
    print(route)
    current_node = route[-1]
    min_node = get_min_node(matrix[current_node], visited)

    if min_node is None:
        return route

    visited.add(current_node)
    return greedy(matrix, visited, route + [min_node])
    

In [7]:
answer = greedy(m)

[0]
[0, 2]
[0, 2, 1]
[0, 2, 1, 3]


In [8]:
answer

[0, 2, 1, 3]

In [9]:
total_cost(m, answer)

17

This is not optimal route but its close to brute force answer. The benifit of the algorithm is relative speed O(n<sup>2</sup>) and simplicity.

In [10]:
total_cost(m, [0, 1, 3, 2]) # route found by brute force

15